# Génération de statistiques pour les passages à l'hopital

Sources: 
- https://www.scansante.fr/ : statistiques aggrégées du PMSI
- http://www.aideaucodage.fr/ : statististiques sur les diagnostics associés, actes CCAM à l'hopital et GHM


Utilisation des données de **2016**


In [1]:
#imports usuels
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rd
import scipy
import zipfile
import json

## On commence par charger quelques informations sur la population

In [2]:
pop_saq = pd.read_csv("../data/pop-sexe-age-quinquennal.zip", header=0, sep=',', encoding="latin_1", dtype={'DR':str,'DR18':str })
pop_saq.dropna(inplace=True) #some cities disappeared leading to none lines

In [3]:
pop_saq.head()

,RR,DR,CR,STABLE,DR18,LIBELLE,ageq_rec01s1rpop2016,ageq_rec01s2rpop2016,ageq_rec02s1rpop2016,ageq_rec02s2rpop2016,...,ageq_rec16s1rpop2016,ageq_rec16s2rpop2016,ageq_rec17s1rpop2016,ageq_rec17s2rpop2016,ageq_rec18s1rpop2016,ageq_rec18s2rpop2016,ageq_rec19s1rpop2016,ageq_rec19s2rpop2016,ageq_rec20s1rpop2016,ageq_rec20s2rpop2016
0,84,01,1,1,01,Abergement-Clémenciat,15.000000,20.000000,30.000000,25.000000,...,5.000000,10.000000,5.000000,25.000000,15.000000,0.000000,0.000000,5.000000,0.000000,0.000000
1,84,01,2,1,01,Abergement-De-Varey,5.000000,20.000000,15.000000,0.000000,...,5.000000,5.000000,10.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,84,01,4,1,01,Ambérieu-En-Bugey,529.591402,529.687146,440.679506,452.831881,...,148.990988,268.453493,160.527009,215.992793,72.838775,157.001158,32.365459,79.016086,8.833555,35.518534
4,84,01,5,1,01,Ambérieux-En-Dombes,25.555140,36.072120,77.130223,40.904082,...,10.414432,15.556395,15.634778,25.892575,5.220346,5.220346,15.556131,10.362045,0.000000,0.000000
5,84,01,6,1,01,Ambléon,5.000000,0.000000,0.000000,0.000000,...,0.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000


In [4]:
pop_saq['RR'].describe()

count    35403.000000
mean        52.300314
std         24.326237
min          1.000000
25%         28.000000
50%         44.000000
75%         76.000000
max         94.000000
Name: RR, dtype: float64

In [5]:
pop_saq['RR'].unique()

array([84, 32, 93, 44, 76, 28, 75, 24, 27, 53, 94, 52, 41, 11,  1,  2,  3,
        4])

In [6]:
pop_saq.dropna(inplace=True) #some cities disappeared leading to none lines
try:
    #remove some a priori useless columns
    del(pop_saq['RR'])
    del(pop_saq['CR'])
    del(pop_saq['DR'])
    del(pop_saq['STABLE'])
except:
    pass

#Gather all the columns in a unique column with variables to describe them ('sex' and 'age')
pop_saq=pop_saq.melt(id_vars=['DR18','LIBELLE'])
tmp=pop_saq['variable'].str.extract(r'ageq_rec(?P<age>\d+)s(?P<sex>\d)rpop2016')
pop_saq=pd.concat([tmp,pop_saq],axis=1)
pop_saq['age']=(pd.to_numeric(pop_saq['age'])-1)*5 #counts for 5 years intervals of ages
del(tmp)
del(pop_saq['variable'])

#rename columns
pop_saq=pop_saq.rename(columns={'LIBELLE':'Ville', 'DR18':'dpt'})
pop_saq['Ville']=pop_saq['Ville'].str.lower()

pop_tot = np.sum(pop_saq["value"])
print("Population totale: ", pop_tot)

#population par département
pop_D=pop_saq.groupby(['dpt']).agg({"value":"sum"})
pop_D= pop_D.reset_index()
pop_D.columns = pop_D.columns.get_level_values(0)

Population totale:  66359521.99452539


In [7]:
#proba of Age, Sex per dpt
pop_AS_D=pop_saq.groupby(["age","sex",'dpt']).agg({"value":"sum"})
pop_AS_D= pop_AS_D.reset_index() #transform the group object into a dataframe
pop_AS_D.columns = pop_AS_D.columns.get_level_values(0)
nb_dpt=pd.merge(pop_AS_D,pop_D,on="dpt")
pop_AS_D['p']=nb_dpt['value_x']/nb_dpt['value_y']

## Modélisation de la probabilité d'aller en séjour hospitalier

Utilisation des données de scan santé sur les nombres de séjours par région, par sexe ou par age.

On dispose des nombres de séjours et également des nombres de patients par région.
- on peut supposer une loi de poisson sur les nombres de passage 
- si on a le nombre total de patient et le nombre de séjour, cela permet de simulter des nombres par patient (loi de Poisson), mais on n'a pas les informations par sexe et age : on peut supposer que les moins âgées vont en moyenne moins de fois à l'hopital que les plus âgés ... 
- le nombre de séjour pour un age donné dépend à la fois du nombre de patients et du nombre de séjours...et on n'a pas les informations pour déméler l'un de l'autre !

NB: on ne garde que les populations de plus de 16 ans


## On commence par regarder au niveau national

In [8]:
mu=2.12
duree_sejour=1+rd.poisson(mu-1,100)
print(duree_sejour)
print( np.mean(duree_sejour) )

[3 2 2 2 3 1 2 3 1 3 3 3 1 1 2 4 2 3 3 2 3 1 4 1 1 2 3 3 1 3 2 3 2 2 3 1 1
 3 3 1 3 4 4 1 2 3 3 1 3 4 2 2 2 2 1 4 2 2 4 3 1 5 2 3 3 2 1 2 3 4 1 2 5 1
 1 3 2 1 2 2 1 2 2 3 2 3 3 2 1 1 2 4 4 2 2 1 1 2 1 1]
2.27


In [9]:
mu=2.29
duree_sejour=1+rd.poisson(mu-1,2175000)
print( np.sum(duree_sejour) )

4980233


In [10]:
sejours=pd.read_csv("../data/PMSI/nbhosp_region.csv",sep=";")[['Dpt',"nbsejours","nbpatients"]]
sejours.rename(columns={"Dpt":"dpt"},inplace=True)
print(sejours.set_index("dpt").loc["35"])
p_sej=pd.merge(sejours,pop_D,on='dpt')
p_sej['p']=p_sej["nbsejours"].astype(float)*p_sej["nbpatients"]/p_sej["value"]
p_sej=p_sej[['dpt',"nbsejours",'p']]
p_sej.set_index("dpt",inplace=True)

nbsejours          2.39
nbpatients    146960.50
Name: 35, dtype: float64


In [11]:
p_sej.loc["35"]


nbsejours    2.390000
p            0.333786
Name: 35, dtype: float64

In [12]:
#exemple de simulation
nb=1000
sim=(1+rd.poisson(p_sej.loc["35"]['nbsejours']-1,nb)) * ( (rd.rand(nb)<p_sej.loc["35"]['p']).astype(int) )
print(sim)
print(np.sum(sim)) #nombre total de séjours hospitaliés
print(np.sum(sim!=0)/nb) #On vérifie la proportion de patients hospitalisés, estimé à 0.334 pour le 35 (cf. ci-dessus)
print(np.sum(sim)/(nb-np.sum(sim==0))) #on vérifie que la durée moyenne des séjours est correcte, estimé à 2.39 pour le 35 (cf. ci-dessus)

[0 0 0 0 3 0 0 0 0 0 3 0 1 0 0 0 2 5 5 0 0 0 0 0 0 0 0 0 3 2 0 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 3 1 3 0 3 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1
 4 0 4 1 0 0 0 0 0 0 0 1 0 2 0 4 0 0 4 7 1 0 0 0 3 0 0 0 2 2 4 2 0 0 0 0 0
 2 0 0 0 3 1 4 0 3 0 3 0 3 0 0 1 0 0 0 0 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 5 0
 2 2 0 2 2 0 2 0 0 0 1 3 1 0 0 0 4 2 2 0 3 0 2 0 4 0 0 0 0 0 0 0 3 2 0 2 0
 0 0 0 2 1 0 0 0 0 0 2 1 0 0 0 2 1 4 0 2 0 6 2 0 4 0 3 0 2 0 4 0 0 0 0 3 2
 0 0 0 2 0 3 1 1 3 2 0 0 3 0 0 0 2 0 0 0 0 0 0 0 2 0 2 0 0 0 3 0 1 1 0 0 0
 0 0 0 0 2 2 1 0 3 0 0 1 4 0 2 0 0 0 0 1 0 1 0 2 0 0 0 4 0 0 0 0 0 0 0 3 4
 0 1 0 2 0 0 0 1 0 1 1 3 2 1 4 3 0 0 3 0 3 1 0 0 0 2 0 0 0 0 0 0 1 0 2 0 0
 0 0 3 0 0 0 0 0 0 4 0 0 0 2 0 0 4 0 0 0 1 1 0 2 0 0 0 2 0 3 0 0 1 0 0 0 1
 1 2 2 0 3 2 0 0 2 3 3 2 0 0 3 3 0 0 2 3 2 3 0 0 3 3 0 0 0 0 0 0 0 1 2 3 0
 0 1 3 0 4 0 0 0 0 0 0 0 3 0 5 1 0 0 0 3 0 1 0 0 0 0 0 2 0 0 0 0 3 0 0 0 0
 0 0 3 1 0 0 0 0 0 0 4 0 0 0 0 2 2 0 0 0 0 2 0 2 2 0 0 0 0 0 2 3 0 3 0 0 5
 0 0 0 3 0 3 0 0 3 0 4 2 

## Maintenant, on regarde pour faire la même procédure, mais en utilisant des données par type de séjour, par age et par sexe

### On commence par la répartition par type de séjours

Plus précisément, on veut : $p(Sej,T|D)$
$$p(Sej,T|D) \approx mean(nbsej|D)\times\frac{count(patients|D)}{count(pop|D)}=\frac{count(nbsej|D)}{count(patients|D)}\times\frac{count(patients|D)}{count(pop|D)}=\frac{count(nbsej|D)}{count(pop|D)}$$


In [13]:
# chargement du tableau de correspondance entre les numéros des régions 
# anciennes régions reconstruites manuellement puisque les stats sont 
# données en utilisant les anciennes régions comme délimiation administrative) 
# et les numéros des départements
reg_dpt=pd.read_csv("../data/PMSI/reg_dpt.csv",sep=";")
reg_dpt.rename(columns={"DPT_COD":"dpt","REG_COD":"reg"},inplace=True)
count_dpts_reg=reg_dpt.groupby(['reg']).agg({"dpt":"count"})
#print(count_dpts_reg)
#reg_dpt

In [14]:
#on définit une fonction qui divide le nombre de séjours par le nombre de département par région pour avoir des comptes par départements
# cette fonction est utilisée à plusieurs reprise par la suite
def norm_dpt(d):
    return d['nbsej']/count_dpts_reg.loc[d['reg']]

In [15]:
type_sej=pd.read_csv("../data/PMSI/nbsejours_type_region.csv",sep=";")[["reg","type","nbsej","DMS","nbpatients"]]
p_sejtype_dpt=pd.merge(reg_dpt, type_sej, how="left", on="reg")
p_sejtype_dpt=pd.merge(p_sejtype_dpt, count_dpts_reg, on="reg",suffixes=('','_count'))
#p_sejtype_dpt['nbpatients']=p_sejtype_dpt['nbpatients']/p_sejtype_dpt['dpt_count']
p_sejtype_dpt=pd.merge(p_sejtype_dpt,pop_D,on='dpt')
p_sejtype_dpt['nbsej']=p_sejtype_dpt['nbsej']/p_sejtype_dpt['dpt_count']
p_sejtype_dpt['p']=p_sejtype_dpt['nbsej']/p_sejtype_dpt['value']
p_sejtype_dpt=p_sejtype_dpt[["reg","dpt","type","DMS",'nbsej',"p"]]

In [16]:
p_sejtype_dpt

,reg,dpt,type,DMS,nbsej,p
0,1,971,0,"4,8",64995.000000,0.164829
1,1,971,1,"0,1",40128.000000,0.101766
2,1,971,2,0,125740.000000,0.318880
3,2,972,0,"5,6",53036.000000,0.140863
4,2,972,1,"0,2",28225.000000,0.074965
...,...,...,...,...,...,...
262,93,83,1,"0,1",113001.166667,0.106990
263,93,83,2,0,160483.666667,0.151946
264,93,84,0,"5,1",157161.333333,0.281307
265,93,84,1,"0,1",113001.166667,0.202264


In [17]:
p_sejtype_dpt[p_sejtype_dpt['dpt']=="22"]

,reg,dpt,type,DMS,nbsej,p
147,53,22,0,"5,5",136589.50,0.227556
148,53,22,1,"0,2",79977.50,0.133242
149,53,22,2,0,136237.75,0.226970


In [18]:
p_sejtype_dpt[p_sejtype_dpt['dpt']=="35"]

,reg,dpt,type,DMS,nbsej,p
153,53,35,0,"5,5",136589.50,0.129803
154,53,35,1,"0,2",79977.50,0.076004
155,53,35,2,0,136237.75,0.129469


### On estime maintenant la proba jointe p(S,A|Sej,T,D)

Pour simplifier, on suppose l'age et le sexe indépendant, et on a alors $p(S,A|Sej,T,D)=p(A|Sej,T,D)*p(S|Sej,T,D)$

Chacun de ces deux termes est obtenu par les données disponibles sur les nombres de séjours par département

In [84]:
#Chargement des statistques sur les sexes
p_sex_sej=pd.read_csv("../data/PMSI/nbsejours_sexe_region.csv",sep=";")[["reg","Type","Sexe","Nb séjours","DMS"]]
p_sex_sej.rename(columns={"Nb séjours":"nbsej", "Type":"type", "Sexe":"sex"}, inplace=True)
p_sex_sej

,reg,type,sex,nbsej,DMS
0,11,0,1,865614,"5,6"
1,44,0,1,104743,"5,6"
2,31,0,1,135217,"5,7"
3,29,0,1,131239,"5,3"
4,24,0,1,182614,"5,8"
...,...,...,...,...,...
157,1,2,2,59951,0
158,2,2,2,36512,0
159,3,2,2,10013,0
160,4,2,2,75765,0


In [20]:
p_sex_sej[p_sex_sej['reg']==53]

,reg,type,sex,nbsej,DMS
12,53,0,1,255340,"5,6"
39,53,0,2,291018,"5,4"
66,53,1,1,155487,"0,2"
93,53,1,2,164423,"0,2"
120,53,2,1,296992,0
147,53,2,2,247959,0


In [21]:
p_sex_sejdpt=pd.merge(reg_dpt, p_sex_sej, how="left", on="reg")
p_sex_sejdpt[p_sex_sejdpt['dpt']=="35"]

,dpt,reg,type,sex,nbsej,DMS
324,35,53,0,1,255340,"5,6"
325,35,53,0,2,291018,"5,4"
326,35,53,1,1,155487,"0,2"
327,35,53,1,2,164423,"0,2"
328,35,53,2,1,296992,0
329,35,53,2,2,247959,0


In [22]:
p_sex_sejdpt['nbsej']=p_sex_sejdpt.apply(norm_dpt,axis=1)
#on estime les probas en utilisant les comptes par départements obtenus précédement
p_sex_sejdpt=pd.merge(p_sex_sejdpt,p_sejtype_dpt[['dpt','type','nbsej']], on=['dpt','type'],suffixes=('','_tot'))
p_sex_sejdpt['p']=p_sex_sejdpt['nbsej']/p_sex_sejdpt['nbsej_tot']
p_sex_sejdpt=p_sex_sejdpt[['reg','dpt','type','sex','DMS','p']]

In [23]:
p_sex_sejdpt

,reg,dpt,type,sex,DMS,p
0,1,971,0,1,"5,3",0.424525
1,1,971,0,2,"4,4",0.575475
2,1,971,1,1,"0,1",0.402985
3,1,971,1,2,"0,1",0.597015
4,1,971,2,1,0,0.523215
...,...,...,...,...,...,...
529,93,84,0,2,"4,9",0.541282
530,93,84,1,1,"0,1",0.478433
531,93,84,1,2,"0,1",0.521567
532,93,84,2,1,0,0.532409


In [24]:
p_sex_sejdpt[p_sex_sejdpt['dpt']=="35"]

,reg,dpt,type,sex,DMS,p
306,53,35,0,1,"5,6",0.467349
307,53,35,0,2,"5,4",0.532651
308,53,35,1,1,"0,2",0.486034
309,53,35,1,2,"0,2",0.513966
310,53,35,2,1,0,0.544988
311,53,35,2,2,0,0.455012


In [25]:
#On fait la même chose pour les ages
p_age_sej=pd.read_csv("../data/PMSI/nbsejours_age_region.csv",sep=";")[["reg","type","age","nbsej","DMS"]]
p_age_sejdpt=pd.merge(reg_dpt, p_age_sej, how="left", on="reg")
p_age_sejdpt['nbsej']=p_age_sejdpt.apply(norm_dpt,axis=1)
#p_age_sejdpt['p']=p_age_sejdpt['nbsej']/np.sum(p_age_sejdpt['nbsej']) #on estime les probas

p_age_sejdpt=pd.merge(p_age_sejdpt,p_sejtype_dpt[['dpt','type','nbsej']], on=['dpt','type'],suffixes=('','_tot'))
p_age_sejdpt['p']=p_age_sejdpt['nbsej']/p_age_sejdpt['nbsej_tot']
p_age_sejdpt=p_age_sejdpt[['reg','dpt','type','age','DMS','p']]

In [26]:
p_age_sejdpt

,reg,dpt,type,age,DMS,p
0,1,971,0,0,"4,4",0.153996
1,1,971,0,17,"3,4",0.074621
2,1,971,0,26,"3,7",0.213940
3,1,971,0,46,"4,3",0.123302
4,1,971,0,56,5,0.222248
...,...,...,...,...,...,...
1864,93,84,2,26,0,0.077556
1865,93,84,2,46,0,0.114032
1866,93,84,2,56,0,0.332595
1867,93,84,2,70,0,0.256652


In [27]:
print(np.sum(p_age_sejdpt[p_age_sejdpt['dpt']=="35"]['p']))
print(np.sum(p_age_sejdpt[(p_age_sejdpt['dpt']=="35") & (p_age_sejdpt['type']==0)]['p'])) # on a bien 1: donc une proba des ages par type !!
p_age_sejdpt[p_age_sejdpt['dpt']=="35"]

3.0000000000000004
1.0


,reg,dpt,type,age,DMS,p
1071,53,35,0,0,"4,2",0.115530
1072,53,35,0,17,"3,1",0.045002
1073,53,35,0,26,"3,7",0.162104
1074,53,35,0,46,"4,2",0.094013
1075,53,35,0,56,"5,1",0.236312
1076,53,35,0,70,"6,5",0.153053
1077,53,35,0,80,"8,6",0.193985
1078,53,35,1,0,"0,2",0.107052
1079,53,35,1,17,"0,2",0.061073
1080,53,35,1,26,"0,2",0.179491


### On en arrive enfin au calcul de la proba d'avoir un séjour

On a des informations qui sont données avec des types de séjours : GHM J ou T (pour les passages à l'hopital en ambulatoire ou à la journée), GHM hors J et T (séjours de plusieurs) ou les séances (type chimio j'imagine) : dans l'immédiat, on ne conserve pas cette information.
- Ce qu'on a plus haut, ce sont bien des $p(A|Sej,T,D)$ où $T$ est le type de séjour (0: hors J et T, 1: T et J, 2: séances), $A$ est la classe d'age, $D$ est le département et $T$ est le type de séjours

En supposant l'indépendance du sexe et de l'age, on a la proba d'avoir un couple (séjour,type) pour une personne d'un certain age, sexe et département donné par :

$$p(Sej,T|S,A,D) = \frac{p(A|Sej,T,D)\times p(S|Sej,T,D)\times p(Sej,T|D)}{p(SA|D)}$$


* $p(Sej,T|D)$ est donné par p_sejtype_dpt
* $p(A|Sej,T,D)$ est donné par p_age_sejdpt qu'on doit affiner
* $p(S|Sej,T,D)$ est donné par p_sex_sejdpt
* $p(SA|D)$ est donné par pop_AS_D (? estimé en population totale ... ou bien en population ayant eu au moins 1 séjour ??)

... on merge toutes ces matrices, et paf, calcul!


On est également intéressé par conserver les informations sur les durées des séjours pour pouvoir donner les dates d'entrée et de sortie.

Il reste alors 1 soucis pour arriver à notre calcul "final", c'est le fait qu'on a pas les mêmes classes d'âge entre les données de population (à 5 ans), au dénominateur, et les données des hospitalisations (et les classes d'ages, sont assez bordéliques dans ce cas!), au numérateur.

Quelques hypothèses supplémentaires :
- uniformité au sein d'une classe d'âges sur les stats du PMSI
- on fait des petits décalages de certaines bornes pour simplifier (et éviter d'avoir à faire des proportions)


In [28]:
#on aligne sur les ages du dénominateurs (classes d'ages de 5 ans)
p_age_sejdpt.reset_index(inplace=True)

#creation d'un tableau avec les entrées souhaitées
ages = np.arange(0,20)*5
dpt = p_age_sejdpt['dpt'].unique()
typef = p_age_sejdpt['type'].unique()
index = pd.MultiIndex.from_product([ages, dpt, typef], names = ["age", "dpt", 'type'])
p_agerefined_sejdpt=pd.DataFrame(index = index).reset_index()

p_age_sejdpt.set_index(['dpt','type','age'],inplace=True)

def augment(x):
    DMS=0
    p=0
    if x['age']<=16:
        DMS = p_age_sejdpt.loc[x['dpt'],x['type'],0]['DMS']
        p = p_age_sejdpt.loc[x['dpt'],x['type'],0]['p']*5/16
    elif 17<=x['age'] and x['age']<=24:
        DMS = p_age_sejdpt.loc[x['dpt'],x['type'],17]['DMS']
        p = p_age_sejdpt.loc[x['dpt'],x['type'],17]['p']*5/8
    elif 25<=x['age'] and x['age']<=44:
        DMS = p_age_sejdpt.loc[x['dpt'],x['type'],26]['DMS']
        p = p_age_sejdpt.loc[x['dpt'],x['type'],26]['p']/4
    elif 45<=x['age'] and x['age']<=54:
        DMS = p_age_sejdpt.loc[x['dpt'],x['type'],46]['DMS']
        p = p_age_sejdpt.loc[x['dpt'],x['type'],46]['p']/2
    elif 55<=x['age'] and x['age']<=69:
        DMS = p_age_sejdpt.loc[x['dpt'],x['type'],56]['DMS']
        p = p_age_sejdpt.loc[x['dpt'],x['type'],56]['p']/3
    elif 70<=x['age'] and x['age']<=79:
        DMS = p_age_sejdpt.loc[x['dpt'],x['type'],70]['DMS']
        p = p_age_sejdpt.loc[x['dpt'],x['type'],70]['p']/2
    elif 80<=x['age']:
        DMS = p_age_sejdpt.loc[x['dpt'],x['type'],80]['DMS']
        p = p_age_sejdpt.loc[x['dpt'],x['type'],80]['p']/4
    return [x['dpt'], x['type'], x['age'], p, DMS]

p_agerefined_sejdpt=p_agerefined_sejdpt.apply(augment, axis=1, result_type='expand')
p_age_sejdpt.reset_index(inplace=True)

p_agerefined_sejdpt.rename(columns={0:'dpt',1:'type',2:'age',3:'p',4:'DMS'},inplace=True)

In [85]:
# on extrait les termes du numérateur dans une seule et même matrice
merge=pd.merge( p_sejtype_dpt[['dpt','type','p']], p_sex_sejdpt[['dpt','sex','type','p']], on=['dpt','type'], suffixes=('_type','_sex') )
merge=pd.merge( merge, p_agerefined_sejdpt[['dpt','type','age','p', 'DMS']], on=['dpt','type'] )
merge.rename( columns={'p':'p_age'},inplace=True )

# puis on ajoute aussi le dénominateur
merge['dpt']=merge['dpt'].str.replace("02B","2B")
merge['dpt']=merge['dpt'].str.replace("02A","2A")
pop_AS_D['sex']=pop_AS_D['sex'].astype(int)

merge=pd.merge( merge, pop_AS_D, on=['dpt','sex','age'] )
merge.rename( columns={'p':'p_pop'},inplace=True )

print(merge.head())

#on fait le calcul
merge['p'] = merge['p_age']*merge['p_sex']*merge['p_type']/merge['p_pop']
p_hosp=merge[['dpt','type','sex','age','p','DMS']]
p_hosp

   dpt  type    p_type  sex     p_sex  age     p_age  DMS         value  \
0  971     0  0.164829    1  0.424525    0  0.048124  4,4  10960.896988   
1  971     1  0.101766    1  0.402985    0  0.025014  0,3  10960.896988   
2  971     2  0.318880    1  0.523215    0  0.000805    0  10960.896988   
3  971     0  0.164829    1  0.424525    5  0.048124  4,4  13307.588357   
4  971     1  0.101766    1  0.402985    5  0.025014  0,3  13307.588357   

      p_pop  
0  0.030415  
1  0.030415  
2  0.030415  
3  0.032610  
4  0.032610  


,dpt,type,sex,age,p,DMS
0,971,0,1,0,0.110717,"4,4"
1,971,1,1,0,0.033728,"0,3"
2,971,2,1,0,0.004417,0
3,971,0,1,5,0.103265,"4,4"
4,971,1,1,5,0.031457,"0,3"
...,...,...,...,...,...,...
10675,84,1,2,90,0.098708,"0,2"
10676,84,2,2,90,0.279012,0
10677,84,0,2,95,0.267124,"7,9"
10678,84,1,2,95,0.097086,"0,2"


In [86]:
p_hosp[(p_hosp['dpt']=="35") & (p_hosp['age']==40) ]

,dpt,type,sex,age,p,DMS
6144,35,0,1,40,0.153662,"3,7"
6145,35,1,1,40,0.103607,"0,2"
6146,35,2,1,40,0.087439,0
6204,35,0,2,40,0.104877,"3,7"
6205,35,1,2,40,0.065610,"0,2"
6206,35,2,2,40,0.043717,0


In [87]:
#on sauvegarde cela
p_hosp.to_csv("p_host.csv")
p_sej.to_csv("p_sej.csv")

## On en tire maintenant une procédure pratique pour la simulation des passages à l'hopital pour un patient

On suppose une population de patients dont on a :
- la localisation (dpt)
- la classe d'age
- le sexe

In [88]:
age=25
sex=2
dpt="35"

On en déduit alors leur probabilité d'aller au moins une fois en séjour hospitalier dans l'année, et une simulation de nombre de séjours hospitaliers

__NB:__ 
* on ne tient finalement pas compte du type d'hospitalisation
* à ce niveau, on n'a pas d'information sur les nombres de séjours par classes d'ages ... ce qui est très dommage (cf difficulté ci-dessus)! On suppose donc que le nombre d'hospitalisation est indépendant de l'age et du sexe des patients

In [89]:
nb=100
phosp=np.sum( p_hosp[(p_hosp['dpt']==dpt) & (p_hosp['age']==age) & (p_hosp['sex']==sex) ]['p'] )
nbhosp=(1+rd.poisson(p_sej.loc[dpt]['nbsejours']-1,nb)) * ( (rd.rand(nb)<float(phosp)).astype(int) )

In [90]:
nbhosp

array([0, 0, 2, 0, 1, 3, 0, 0, 2, 0, 0, 4, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0,
       0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,
       0, 1, 0, 0, 0, 5, 0, 1, 0, 3, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [97]:
float(p_hosp[ (p_hosp['dpt']==dpt) & (p_hosp['age']==age) & (p_hosp['sex']==sex) & (p_hosp['type']==0) ]['DMS'])

ValueError: could not convert string to float: '3,7'

In [106]:
ps=pd.read_csv("p_sej.csv")
psmail
ps


,nbsejours,p
dpt,,
971,3.29,0.575186
972,2.98,0.456774
973,2.00,0.245094
974,2.64,0.410869
75,2.29,0.284315
...,...,...
43,2.27,0.637381
63,2.27,0.222915
13,2.38,0.210245


## Utilisation des données des codes CIM des passages à l'hopital

Maintenant qu'on sait combien de fois un patient vµa à l'hopital, il faut savoir ce qui s'y passe ... pour cela, il faut générer des codes CIM qui caractérisent les patients.

* données de scan santé: les nombres d'hospitalisation pour chaque code CIM
    * attention, il y a des code CIM dits primaires (qui peuvent servir au diagnostic principal) et des codes CIM autres pour les codes 
    * les comptes ne sont donnés que pour les diagnostics 
    * 
* aucune information sur la typologie des patients à qui sont "délivrés" des CIMs ... on donne donc de manière uniforme à tous les patients
* données de aide au codage, permettent de déterminer les diagnostics liés les GHM et même les actes CCAM associés

### On commence par fusionner les deux sources de données dont on dispose
* les co-administrations pourront servir à générer les GHM ou les diagnostics associés/relatifs
    * *NB* la collecte de données a omis la collecte des diagnostics relatifs ... on a que les diagnostics associés !! À finaliser ...
    * *NB* peut être collecter l'information de si un code peut servir de diagnostic principal (sinon, a priori les comptes sont là pour cela)
* les nombres de code CIM utilisés comme code diagnostic

In [35]:
with open("../data/PMSI/result.json") as f:
    result=json.load(f)

#données associées au codes servant au diagnostic principal
result["A010"]

{'ccam': {'ZCQM008': 100.0,
  'ZBQK002': 65.747663551402,
  'YYYY600': 28.598130841121,
  'DZQM006': 15.654205607477,
  'ZCQM006': 6.495327102803699,
  'DEQP003': 0.8411214953271,
  'GLHF001': 0.2336448598130998,
  'ZCQK002': 0.18691588785050017,
  'ZBQH001': 0.18691588785050017,
  'ACQK001': 0.1401869158879001,
  'JVJF004': 0.09345794392520013,
  'GLLD017': 0.09345794392520013,
  'YYYY015': 0.09345794392520013},
 'cim': {'Z29.0': 100.0,
  'A02.1': 80.918799113439,
  'A09.0': 68.285311303647,
  'A02.0': 63.771912149909,
  'R74.0': 52.931694539593,
  'R65.0': 45.214587950836,
  'U83.2': 44.731009470079,
  'E86': 41.386258311505,
  'R50.9': 38.36389280677,
  'A01.0': 29.881120290147,
  'A09.9': 23.15131976627,
  'B18.2': 23.010276042716,
  'E87.68': 17.1065887568,
  'N14.1': 15.575256901067998,
  'R50.8': 14.547652629458,
  'Y43.4': 14.346161595809,
  'Z94.1': 14.124521458795002,
  'G93.4': 12.190207535765,
  'U88': 11.847672778561,
  'R39.2': 11.767076365102},
 'ghm': {'18M102': 100.0,


In [36]:
cims=pd.read_csv("../data/PMSI/cimcounts_all.csv")[['cim', 'count']]
cims.set_index("cim", inplace=True)
cims.head()

,count
cim,
A000,0
A009,0
A010,139
A011,0
A012,0


In [37]:
#on ajoute le compte à l'intérieur
tot_cims=0
for cim,val in result.items():
    try:
        val['count']=int(cims.loc[cim]['count'])
    except KeyError:
        val['count']=0
    tot_cims += int(val['count'])
result["A010"]

{'ccam': {'ZCQM008': 100.0,
  'ZBQK002': 65.747663551402,
  'YYYY600': 28.598130841121,
  'DZQM006': 15.654205607477,
  'ZCQM006': 6.495327102803699,
  'DEQP003': 0.8411214953271,
  'GLHF001': 0.2336448598130998,
  'ZCQK002': 0.18691588785050017,
  'ZBQH001': 0.18691588785050017,
  'ACQK001': 0.1401869158879001,
  'JVJF004': 0.09345794392520013,
  'GLLD017': 0.09345794392520013,
  'YYYY015': 0.09345794392520013},
 'cim': {'Z29.0': 100.0,
  'A02.1': 80.918799113439,
  'A09.0': 68.285311303647,
  'A02.0': 63.771912149909,
  'R74.0': 52.931694539593,
  'R65.0': 45.214587950836,
  'U83.2': 44.731009470079,
  'E86': 41.386258311505,
  'R50.9': 38.36389280677,
  'A01.0': 29.881120290147,
  'A09.9': 23.15131976627,
  'B18.2': 23.010276042716,
  'E87.68': 17.1065887568,
  'N14.1': 15.575256901067998,
  'R50.8': 14.547652629458,
  'Y43.4': 14.346161595809,
  'Z94.1': 14.124521458795002,
  'G93.4': 12.190207535765,
  'U88': 11.847672778561,
  'R39.2': 11.767076365102},
 'ghm': {'18M102': 100.0,


In [38]:
print(tot_cims)

21872936


In [39]:
#sélection uniquement des cims avec des comptes non-nuls
cim_stats = {r:v for r,v in result.items() if v['count']!=0}

cim_stats["A020"]

{'ccam': {'HJQE001': 100.0,
  'ZCQM008': 77.352472089314,
  'ZCQK002': 61.40350877193,
  'HHQE004': 43.955342902711,
  'HZHE001': 43.125996810207,
  'ZZQM005': 36.810207336523,
  'ZCQM006': 35.215311004785,
  'JAQM004': 34.385964912281,
  'ZBQK002': 25.933014354067,
  'JAQM003': 21.499202551834,
  'YYYY600': 11.802232854864},
 'cim': {'A02.0': 100.0,
  'E86': 99.32850326913,
  'Z29.0': 93.620781056724,
  'A08.0': 89.503445838487,
  'A09.0': 79.501678741827,
  'U83.2': 77.64622724863,
  'I88.0': 70.489485774872,
  'A08.2': 69.305531012546,
  'E87.68': 68.050892383813,
  'R39.2': 65.842021558579,
  'U82.0': 65.18819579431,
  'E87.18': 64.322318430818,
  'R50.8': 61.742357306945,
  'R56.0': 60.134299346174,
  'R65.0': 59.144725216469,
  'R50.9': 57.978441420746,
  'A02.1': 55.133415797844,
  'R82.4': 53.083583672027,
  'R57.1': 51.104435412617},
 'ghm': {'06M021': 100.0,
  '06M022': 95.827019466138,
  '06M032': 83.273108753934,
  '06M031': 76.477444923651,
  '06M033': 70.963981816062,
  '

In [40]:
# et on sauvegarde cela
with open("../data/PMSI/cim_stats.json","w") as f:
    json.dump(cim_stats,f)

In [41]:
counts={i:[s['count']] for i,s in cim_stats.items()}
counts=pd.DataFrame.from_dict(counts).transpose()
counts.rename(columns={0:"count"}, inplace=True)
counts.head()

,count
A010,139
A020,2996
A021,228
A022,40
A028,79


In [42]:
counts.loc["A021"]['count']

228

In [43]:
np.sum(counts['count'])

21872936

In [44]:
counts['p']=counts['count']/np.sum(counts['count'])

In [45]:
counts.head()

,count,p
A010,139,0.000006
A020,2996,0.000137
A021,228,0.000010
A022,40,0.000002
A028,79,0.000004


In [46]:
#exemple de génération d'une collection de codes CIM en repartition égale à leurs probabilités
list(counts.reset_index().sample(n=10, replace=True, weights='p', random_state=1)['index'])

['N421',
 'Z491',
 'A020',
 'K409',
 'H259',
 'F320',
 'I442',
 'K650',
 'M751',
 'S618']

In [49]:
#Génération d'une collection de DAS de "A020"
cim_stats["A020"]["cim"]
counts={i:s for i,s in cim_stats["A020"]["cim"].items()}
counts=pd.DataFrame.from_dict(counts).transpose()

ValueError: If using all scalar values, you must pass an index

In [65]:
counts=pd.DataFrame.from_dict( dict(cim_stats["A020"]["cim"]), orient='index' )
counts.rename(columns={0:"p"},inplace=True)
counts['p']=counts['p']/np.sum(counts['p'])
list(counts.sample(n=4,weights='p').reset_index()['index'])

['A02.0', 'I88.0', 'R57.1', 'Z29.0']

In [75]:
counts=pd.DataFrame.from_dict( dict(cim_stats["A020"]["cim"]), orient='index' )
counts[0]=counts[0]/np.sum(counts[0])
list(counts.sample(n=4,weights=counts[0]).reset_index()['index'])

['A02.0', 'I88.0', 'A02.1', 'A09.0']

### Quelques commentaires sur la génération obtenue

* Pour faire une simulation ... il n'y a plus qu'à prendre les codes en ordre de cette liste
    * une amélioration possible serait de "répartir" cette liste par profils de patients (en particulier pour des codes chimios)!
    * considère des statistiques que pour les diagnotics principaux
* Beaucoup de code Z511 correspondant à des séances de chimiothérapie -> normalement, c'est des codes qui devraient être réservés à des patients avec des ALD correspondantes !
* Il me manque les diagnostics reliés (j'ai juste des diagnostics associés)